<a href="https://colab.research.google.com/github/jtneumann/MLwork/blob/master/fastai/ml1/Titanic3sub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q fastai==0.7.0
!pip install -q pandas==0.23.4

In [0]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [0]:
import seaborn as sns
sns.set_style('whitegrid')

In [0]:
import pandas as pd
df_raw = pd.read_csv('train.csv', low_memory=False)

In [6]:
df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
train_cats(df_raw)

In [0]:
df,y,nas=proc_df(df_raw, 'Survived')

In [9]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na
0,1,3,109,2,22.0,1,0,524,7.2500,0,3,False
1,2,1,191,1,38.0,1,0,597,71.2833,82,1,False
2,3,3,354,1,26.0,0,0,670,7.9250,0,3,False
3,4,1,273,1,35.0,1,0,50,53.1000,56,3,False
4,5,3,16,2,35.0,0,0,473,8.0500,0,3,False


In [10]:
m=RandomForestClassifier(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9809203142536476

In [11]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

n_valid = 209
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((682, 12), (682,), (209, 12))

In [0]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res=[rmse(m.predict(X_train),y_train), rmse(m.predict(X_valid), y_valid),
         m.score(X_train, y_train), m.score(X_valid, y_valid)]
    
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [13]:
m = RandomForestClassifier(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 27.3 ms, sys: 2.27 ms, total: 29.5 ms
Wall time: 112 ms
[0.10830607221477646, 0.43197554423068574, 0.9882697947214076, 0.8133971291866029]


In [14]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1, max_depth=3, bootstrap=False)
m.fit(X_train, y_train)
print_score(m)

[0.4106357277195512, 0.39735970711951313, 0.8313782991202346, 0.8421052631578947]


In [15]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=1, bootstrap=False)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.6030226891555273, 1.0, 0.6363636363636364]


In [16]:
m =RandomForestClassifier(n_estimators=5, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.1624591083221647, 0.4482822077654243, 0.9736070381231672, 0.7990430622009569]


best so far

In [17]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.03829197905337418, 0.36602088993120147, 0.998533724340176, 0.8660287081339713]


In [18]:
m = RandomForestClassifier(n_estimators=60, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.33886949683494305, 1.0, 0.8851674641148325]


In [19]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.3594253787238923, 1.0, 0.8708133971291866, 0.8137829912023461]


In [20]:
m = RandomForestClassifier(n_estimators=200, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.3244428422615251, 1.0, 0.8947368421052632, 0.8167155425219942]


In [21]:
X_train, X_valid = split_vals(df, n_trn)
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.24816038707378332, 0.3527065549087486, 0.9384164222873901, 0.8755980861244019, 0.8079178885630498]


In [0]:
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

In [23]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.3851303141167605, 1.0, 0.8516746411483254, 0.8269794721407625]


In [24]:
t = m.estimators_[0].tree_
dectree_max_depth(t)

13

New best

In [25]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.27612738783186724, 0.3458572319330373, 0.9237536656891495, 0.8803827751196173, 0.8167155425219942]


In [26]:
m = RandomForestClassifier(n_estimators=100, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.0, 0.33886949683494305, 1.0, 0.8851674641148325, 0.8064516129032258]


In [27]:
m = RandomForestClassifier(n_estimators=40, n_jobs=-1, min_samples_leaf=3, oob_score=True, max_features=0.5)
m.fit(X_train, y_train)
print_score(m)

[0.2625165823557251, 0.3527065549087486, 0.9310850439882697, 0.8755980861244019, 0.8152492668621701]


In [28]:
fi=rf_feat_importance(m,df); fi

,cols,imp
3,Sex,0.339582
7,Ticket,0.120758
8,Fare,0.108126
2,Name,0.098792
0,PassengerId,0.084658
4,Age,0.078252
1,Pclass,0.074950
9,Cabin,0.038755
5,SibSp,0.027459
10,Embarked,0.014807


Final Model:

In [29]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.26804385337361925, 0.3788676152120412, 0.9281524926686217, 0.8564593301435407, 0.8152492668621701]


In [31]:
df_test=pd.read_csv('test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
train_cats(df_test)

In [0]:
df_test, _,nas=proc_df(df_test)

In [34]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na,Fare_na
0,892,3,207,2,34.5,0,0,153,7.8292,0,2,False,False
1,893,3,404,1,47.0,1,0,222,7.0000,0,3,False,False
2,894,2,270,2,62.0,0,0,74,9.6875,0,2,False,False
3,895,3,409,2,27.0,0,0,148,8.6625,0,3,False,False
4,896,3,179,1,22.0,1,1,139,12.2875,0,3,False,False


In [35]:
df_test.shape

(418, 13)

In [40]:
df_test.drop(['Fare_na'], axis=1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na
0,892,3,207,2,34.5,0,0,153,7.8292,0,2,False
1,893,3,404,1,47.0,1,0,222,7.0000,0,3,False
2,894,2,270,2,62.0,0,0,74,9.6875,0,2,False
3,895,3,409,2,27.0,0,0,148,8.6625,0,3,False
4,896,3,179,1,22.0,1,1,139,12.2875,0,3,False
5,897,3,367,2,14.0,0,0,262,9.2250,0,3,False
6,898,3,85,1,30.0,0,0,159,7.6292,0,2,False
7,899,2,58,2,26.0,1,1,85,29.0000,0,3,False
8,900,3,5,1,18.0,0,0,101,7.2292,0,1,False
9,901,3,104,2,21.0,2,0,270,24.1500,0,3,False


In [43]:
survived = m.predict(df_test)

ValueError: ignored

In [44]:
df_test.shape

(418, 13)

In [45]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na,Fare_na
0,892,3,207,2,34.5,0,0,153,7.8292,0,2,False,False
1,893,3,404,1,47.0,1,0,222,7.0000,0,3,False,False
2,894,2,270,2,62.0,0,0,74,9.6875,0,2,False,False
3,895,3,409,2,27.0,0,0,148,8.6625,0,3,False,False
4,896,3,179,1,22.0,1,1,139,12.2875,0,3,False,False


In [0]:
df_test.drop(['Fare_na'], axis=1, inplace=True)

In [47]:
df_test.shape

(418, 12)

In [0]:
survived = m.predict(df_test)

In [49]:
survived

array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 

In [52]:
len(survived)

418

In [0]:
df_test['Survived'] = survived

In [0]:
answer = df_test[['PassengerId','Survived']]

In [58]:
answer.shape

(418, 2)

In [59]:
answer.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [0]:
Results = answer.to_csv('Results3.csv')

In [0]:
Results